<a href="https://colab.research.google.com/github/dpatel200/BIOL3340_Fall2025/blob/main/BIOL3340_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Final project (Fall-2025)**